In [8]:
# Bibliotecas
import pandas as pd

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

from tensorflow.keras.layers import Dense, Dropout, Activation, Input
from tensorflow.keras.models import Model

In [9]:
# Lendo dados;
data = pd.read_csv("../Databases/games.csv")

In [10]:
# Filtros;
data = data[data["NA_Sales"] > 1]
data = data.loc[data["EU_Sales"] > 1]

# Apagando colunas irrelevantes;
data = data.drop(["Other_Sales", "NA_Sales","EU_Sales","JP_Sales", "Developer","Name"], axis=1)

# Apagando linhas com valores faltantes;
data = data.dropna(axis=0)



In [11]:
# Divisão da base de dados;
X = data.iloc[:,[0, 1, 2, 3, 5, 6, 7, 8, 9]].values
sales_global = data.iloc[:, 4].values

# Transformando categoricas para numerico;
encoder = LabelEncoder()
categorical = [0, 2, 3, 8]
for i in categorical:
    X[:,i] = encoder.fit_transform(X[:,i])

# Transformando pra p formato oneHot
onehotencorder = ColumnTransformer(transformers=[("OneHot", OneHotEncoder(), categorical)],remainder='passthrough')
X = onehotencorder.fit_transform(X).toarray()

### Criando rede neural

In [12]:
# Criando camadas;
#--camada de entrada;
input_layer = Input(shape=(61,))
#--camada Ocultas;
hidden_layer_1 = Dense(units=32, activation=Activation("sigmoid"))(input_layer)
hidden_layer_2 = Dense(units=32, activation=Activation("sigmoid"))(hidden_layer_1)
#--camada de saídas;
out_layer_1 = Dense(units=1, activation=Activation("linear"))(hidden_layer_2)

# Criando Regressor;
Regressor = Model(inputs= input_layer,
              outputs=[out_layer_1])
Regressor.compile(optimizer = "adam", loss="mse")

# Treinamento
Regressor.fit(X, [sales_global], epochs=5000, batch_size=100)

3/3 [==============================] - 0s 998us/step - loss: 12.8877
Epoch 4762/5000
3/3 [==============================] - 0s 2ms/step - loss: 17.6213
Epoch 4763/5000
3/3 [==============================] - 0s 1ms/step - loss: 16.0815
Epoch 4764/5000
3/3 [==============================] - 0s 1ms/step - loss: 12.4750
Epoch 4765/5000
3/3 [==============================] - 0s 1ms/step - loss: 12.9042
Epoch 4766/5000
3/3 [==============================] - 0s 2ms/step - loss: 12.2799
Epoch 4767/5000
3/3 [==============================] - 0s 1ms/step - loss: 12.1362
Epoch 4768/5000
3/3 [==============================] - 0s 1ms/step - loss: 15.1634
Epoch 4769/5000
3/3 [==============================] - 0s 1ms/step - loss: 14.3596
Epoch 4770/5000
3/3 [==============================] - 0s 1ms/step - loss: 12.7519
Epoch 4771/5000
3/3 [==============================] - 0s 1ms/step - loss: 15.9289
Epoch 4772/5000
3/3 [==============================] - 0s 1ms/step - loss: 12.6341
Epoch 4773/5000
3/

In [13]:
# Previsão apenas para demonstração
pred_global = Regressor.predict(X)